In [62]:
from math import sqrt
import numpy as np
import pandas as pd

In [63]:
def get_data(filename):
    data = pd.read_csv(filename)
    dataset = data.sample(frac = 1).reset_index(drop = True)
    return dataset

data_set = get_data("/home/osboxes/Desktop/Iris.csv")
n = len(data_set)
train_len = int(0.8*n)

train_set = data_set[0:train_len]
test_set = data_set[data_set.columns[0:4]][train_len:n]

In [64]:
def calculate_distance(point, point0):
    distance = 0
    for i in range(len(point)-1):
        distance += (point[i] - point0[i])**2
    return sqrt(distance)


In [65]:
def get_neighbours(train_set, point0, k):
    distances = []
    
    for point in train_set:
        distance = calculate_distance(point0, point)
        distances.append((point, distance))
    #print(distances)
    distances.sort(key = lambda x: x[1])
    #print(distances)
    neighbours = [point[0] for point in distances[0:k]]
    return neighbours

In [68]:
def predict(train_set, point0, k):
    neighbours = get_neighbours(train_set, point0, k)
    values = [point[-1] for point in neighbours]
    prediction = max(values, key = values.count)
    return prediction

In [71]:
train_list = train_set.values.tolist()
test_list = test_set.values.tolist()

for point0 in test_list:
    print(predict(train_list, point0, 5))

2.0
1.0
3.0
1.0
2.0
3.0
3.0
2.0
1.0
1.0
3.0
1.0
2.0
1.0
1.0
2.0
2.0
3.0
3.0
3.0
2.0
2.0
3.0
3.0
1.0
2.0
1.0
3.0
3.0
1.0
